google sheet template properties?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var renderRows = importer.import('google sheet layout template');
var getRows = importer.import('get worksheet rows');

function getTemplateProperties(value, properties, templates) {
    var returnObj = {};
    var promise;
    if(typeof properties[value + '-data'] != 'undefined') {
        promise = Promise.resolve(properties[value + '-data']);
    } else {
        promise = getDataSheet(templates[value].data);
    }
    return promise
        .then(data => {
            properties[value + '-data'] = data;
            returnObj[value + '-data'] = data;
            var templatePromise;
            if(typeof properties[value] != 'undefined') {
                templatePromise = Promise.resolve(properties[value]);
            } else {
                templatePromise = getSections(value, properties, templates)
                    .then(rows => renderRows(value, rows, properties));
            }
            return templatePromise;
        })
        .then(template => {
            properties[value] = template;
            returnObj[value] = template;
            return returnObj;
        });
}

// must do this up front so we can process all data
function getSections(layoutTitle, properties, templates) {
    var worksheet = templates[layoutTitle].template;
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return getRows(cells)
        .then(rows => {
            var sections = rows.reduce((obj, row, i) => {
                var subsections = row.reduce((obj, c, j) => {
                    var value = row[j + 1];
                    if(c === ':section' && typeof templates[value] != 'undefined') {
                        obj[value] = getTemplateProperties(value, properties, templates);
                    }
                    return obj;
                }, obj);
                return obj;
            }, {});
            return Promise.all(Object.values(sections)).then(() => rows);
        })
}

module.exports = getTemplateProperties;

google sheet layout template?

Convert a google sheet page to an HTML template



In [ ]:
var importer = require('../Core');
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    var dataKeys = (data || []).reduce((keys, cur) => {
        // get all keys in data
        return keys.concat(Object.keys(cur));
    }, []).filter((k, h, a) => a.indexOf(k) == h);
    var dataClasses = dataKeys
        .filter(k => ('\\{\\{' + k + '\\}\\}', 'ig').match(c))
        .map(k => 'val-' + k)
        .join(' ')
    return dataClasses;
}

function renderRows(layoutTitle, rows, properties) {
    // set object properties for mustache template
    var html = rows.reduce((arr, row, i) => {
        var rowsHtml = row.reduce((arr, c, j) => {
            if(c.substr(0, 1) === ':') {
                // use subsequent column for property values
                var value = row[j + 1];
                if(c.substr(1).toLowerCase() === 'section') {
                    arr[arr.length] = `
<div class="section-${value}">
${properties[value]}
</div>
`;
                } else {
                    properties[c.substr(1)] = value;
                }
            // render markdown content if it is not the value for the previous property
            } else if(j == 0 || j >= 1 && row[j - 1]
                      && row[j - 1].substr(0, 1) !== ':') {
                var dataClasses = getDataClasses(c, properties[layoutTitle + '-data']);

                var mdHtml = md.render(c);
                // if all markdown did was insert a paragraph and line break, use value instead
                if(mdHtml.replace(/<\/?p>|\s+/ig, '') == c.replace(/<\/?p>|\s+/ig, '')) {
                    mdHtml = c;
                }
                arr[arr.length] = `
<div class="${dataClasses} cell-${j}">
${mdHtml}
</div>
`;
            }
            return arr;
        }, []);

        // render mustache templates
        if(rowsHtml.length > 0) {
            var rowTemplate = `
<div class="${properties['class'] || ''} col-${rowsHtml.length}">
${rowsHtml.join('')}
</div>
`;
            arr[arr.length] = rowTemplate;
        }

        return arr;
    }, []);

    // render mustache page template
    return html.join('');
}

module.exports = renderRows;

test google sheet template?


In [11]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var renderRows = importer.import('google sheet layout template');
var getTemplateProperties = importer.import('google sheet template properties');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get html from template', () => {
    
    it('should process at least one template', () => {
        var templates, key, properties = {};
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getDataSheet(templates[key].data);
            })
            .then(data => {
                assert(typeof key != 'undefined', 'should have at least one template');
                properties[key + '-data'] = data;
                return getTemplateProperties(key, properties, templates);
            })
            .then(() => {
                console.log(properties[key]);
                console.log(properties);
                assert(properties[key].length > 0, 'should have a page called template');
            })
    })
    
    
})


reading notebook /Users/briancullinan/jupytangular2/Utilities/google sheets.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/google sheets.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Marketing/convert spreadsheet.ipynb
compiling /Users/briancullinan/jupytangular2/Marketing/convert spreadsheet.ipynb[1]
reading notebook /Users/briancullinan/jupytangular2/Marketing/convert spreadsheet.ipynb
compiling /Users/briancullinan/jupytangular2/Marketing/convert spreadsheet.ipynb[0]


ReferenceError: describe is not defined

output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');
var Mustache = require('mustache');

function wrapTemplate(layoutTitle, html, properties) {
    // TODO: turn this in to an array/section?
    var script = '', banner = '', domain = '';
    if(typeof properties['stylesheet'] == 'undefined') {
        properties['stylesheet'] = '/illuminati-style.css';
    }
    if(typeof properties['script'] != 'undefined') {
        script = '<script src="{{script}}"></script>';
    }
    if(typeof properties['banner'] != 'undefined') {
        banner = 'background-image: url({{{banner}}});';
    }
    if(typeof properties['domain'] != 'undefined') {
        domain = properties['domain'].includes(':') ? '{{domain}}' : ('https://{{domain}}');
    }
    
    // TODO: automatically set title if it isn't set manually
    var result;
    if(typeof properties['title'] == 'undefined' && (result = (/<h1>(.*)<\/h1>/ig).exec(html))) {
        properties['title'] = result[1];
    }
    
    var pageHtml = `
<!doctype html>
<html>
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
<link rel="stylesheet" media="screen" href="{{stylesheet}}">
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<link rel="canonical" href="${domain}/${layoutTitle}">
<title>{{title}}</title>
<style>
body > div.col-1:nth-of-type(2) {
${banner}
}
</style>
</head>
<body class="${layoutTitle.replace(/\//ig, ' ')} {{class}}">
${html}
${script}
</body>
</html>`;
    Mustache.parse(pageHtml);
    return Mustache.render(pageHtml, properties);
}

module.exports = wrapTemplate;


test output sheet template?


In [ ]:
var fs = require('fs');
var path = require('path');
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getTemplateProperties = importer.import('google sheet template properties');
var wrapTemplate = importer.import('output google sheet template');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('output rendered html', () => {
    
    it('should process at least one template', () => {
        var templates, key, properties = {};
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getTemplateProperties(key, properties, templates)
                    .then(() => wrapTemplate(key, properties[key], properties))
            })
            .then(page => {
                assert(page.length > 0, 'should have a page');
            })
    })
    
    
})


collect google sheets resources?


In [ ]:
var fs = require('fs');
var path = require('path');
var {JSDOM, XPathResult} = require('jsdom');
var {Readable} = require('stream');
var importer = require('../Core');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var getArrayXPath = importer.import('get xpath array');
var streamToGoogle = importer.import('upload files google cloud');

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m; while ((m = ex.exec(str)) && co.push(m[i])); return co;
};

// detect links and write out every part of the site
function collectTemplateResources(layoutTitle, page, properties, templates, bucketName, rendered, sTG) {
    if(sTG) {
        streamToGoogle = sTG;
    }
    if(!rendered) {
        rendered = [];
    }
     
    var stream = new Readable();
    stream.push(page);
    stream.push(null);
    
    // get all images and urls from template
    var dom = new JSDOM(page);
    
    var linksObjs = getArrayXPath('//a[@href]', dom.window.document.body);
    var links = linksObjs.map(l => l.getAttribute('href'));
    
    var imgObjs = getArrayXPath('//img[@src]', dom.window.document.body);
    var imgs = imgObjs.map(l => l.getAttribute('src'));
    
    var stylesObjs = getArrayXPath('//link[@href]', dom.window.document.body);
    var styles = stylesObjs.map(l => l.getAttribute('href'));

    var scriptsObjs = getArrayXPath('//script[@src]', dom.window.document.body);
    var scripts = scriptsObjs.map(l => l.getAttribute('src'));
    
    var backgrounds = regexToArray(/url\((.*?)\)/ig, page, 1);
    
    var paths = [];
    
    // TODO: return as a list and break into separate fuunction
    // copy resource images to output directory
    var searches = imgs.concat(styles).concat(backgrounds).concat(scripts)
        .filter((cur, i, arr) => arr.indexOf(cur) == i && rendered.indexOf(cur) === -1)
        .map(i => {
            rendered[rendered.length] = i;
            return i;
        });
    
    var linked = links
        .filter((cur, i, arr) => arr.indexOf(cur) == i && !cur.includes(':'))
        .map(l => {
            var urlParts = l.split('/')
            var key = urlParts[1];
            var fileName = key;
            if(rendered.indexOf(l) > -1 || typeof templates[key] == 'undefined') {
                return Promise.resolve(false);
            }
            
            console.log('Rendering ' + l);
            rendered[rendered.length] = l;
            return getTemplateProperties(key, properties, templates)
                .then(() => {
                    // if rendering a page has url parts
                    if(urlParts.length > 2) {
                        // use the third argument to find the correct row in the data set
                        var rowData = properties[key + '-data']
                            .filter(row => row['link'] == l || row['url'] == l)[0];
                        Object.assign(properties, rowData);
                        fileName = l.substr(1);
                    }
                    return wrapTemplate(fileName, properties[key], properties)
                })
                .then(page => collectTemplateResources(fileName, page, properties, templates, bucketName, rendered));
        });
    return streamToGoogle(layoutTitle + '.html', bucketName, stream)
        .then(() => Promise.all(linked))
        .then(() => rendered);
}

module.exports = collectTemplateResources;

test google sheets resources?


In [ ]:
var assert = require('assert');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');
var findSimilarFile = importer.import('find similar file');

//https://docs.google.com/spreadsheets/d/1hVchh-0CniUy427vpukwqPyeXVGKNc9SyBztjxlzbcM/copy
//var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';
var DOWNLOAD_DIR = path.resolve(path.join(__dirname, '../../Downloads'));
var docsId = '1QeZ3WduNFmjtNf_Q3Xe5zAh8dzP_nn7nr-AQU0_hXg8';
var bucketName = 'sheet-to-web.com';
//var bucketName = 'www.shopilluminati.com';

function streamToOutput(fileUrl, bucketName, stream) {    
     return new Promise((resolve, reject) => {
        if(typeof stream == 'object') {
            var outputPath = path.join(path.resolve('./.output/'), fileUrl.replace(/\?.*/ig, ''));
            if(!fs.existsSync(path.dirname(outputPath))) {
                fs.mkdirSync(path.dirname(outputPath));
            }
            var writeStream = fs.createWriteStream(outputPath);
            stream.pipe(writeStream)
            .on("error", (err) => {
                reject(err);
            })
            .on('finish', () => {
                resolve(outputPath);
            });
        } else {
            var outputPath = path.join(path.resolve('./.output/'), path.basename(fileUrl.replace(/\?.*/ig, '')));
            var writeStream = fs.createWriteStream(outputPath);
            (fileUrl.includes('https://') ? https : http).get(fileUrl, response => {
                response.pipe(writeStream)
                .on("error", (err) => {
                    reject(err);
                })
                .on('finish', () => {
                    resolve(outputPath);
                });
            }).catch(e => console.log(e));
        }
     });
}

describe('output template resources', () => {

    it('should process at least one template', () => {
        var templates, key, properties = {};
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getTemplateProperties(key, properties, templates)
                    .then(() => wrapTemplate(key, properties[key], properties));
            })
            .then(page => {
                // TODO: move this in to a new function for detecting links and writing every part of site
                assert(page.length > 0, 'should have a page');
                return collectTemplateResources(key, page, properties, templates, bucketName, false, streamToOutput)
                    .then(resources => {
                        console.log(resources);
                        return Promise.all(resources.filter(r => !r.includes(':') && r.includes('.'))
                            .map(r => {
                                return findSimilarFile(path.basename(r), DOWNLOAD_DIR)
                                    .then(f => {
                                        if(f) {
                                            return streamToGoogle(
                                                f,
                                                bucketName,
                                                fs.createReadStream(path.join(DOWNLOAD_DIR, f)));
                                        } else {
                                            return Promise.resolve(false);
                                        }
                                    });
                            }));
                    });
            })
    }).timeout(60000)
    
    
})


get sheet identifier from link?


In [4]:
var util = require('util');
var getInfo = importer.import('get google sheet info');
var getDataSheet = importer.import('google sheet array objects');
var addRow = importer.import('add row data google sheet');
var uuid = require('uuid/v1');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var project = 'spahaha-ea443';

function safeName(name) {
    return name.replace(/[^a-z\-]/ig, '-').substr(0, 40);
}

function getSheet(link) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var name, title;
    
    return getInfo(link)
        // return assigned subdomain
        .then(info => {
            name = safeName(info.properties.title.replace(/\s*(config|configuration)\s*/ig, '')) + '-' + uuid().substr(0, 5);
            title = info.properties.title;
            return getDataSheet(purchaseId, 'Purchases');
        })
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(match) {
                console.log(`Purchase ${docId} already exists: ${match.domain} or ${match.bucket}`);
                return Promise.resolve(match.domain || match.bucket);
            }
            return addRow(purchaseId, {
                timestamp: Date.now(),
                title: title,
                email: '',
                address: '',
                domain: '',
                bucket: name + '.sheet-to-web.com',
                project: project,
                sheet: docId
            })
                .then(() => name + '.sheet-to-web.com')
        })
}

module.exports = getSheet;

reading notebook /Users/briancullinan/jupytangular2/Utilities/google sheets.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/google sheets.ipynb[2]
reading notebook /Users/briancullinan/jupytangular2/Utilities/google sheets.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/google sheets.ipynb[4]


[Function: getSheet]

test get sheet identifier?



In [3]:
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');

describe('get sheet identifier', () => {

    it('should process the sheet id', () => {
        return getSheet('https://docs.google.com/spreadsheets/d/113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k/edit#gid=0')
            .then(s => {
                console.log(s);
                assert(s.length > 0, 'sheet domain should be returned')
            })
    })
    
})


ReferenceError: describe is not defined

google sheet handler?


In [2]:
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');

async function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return await getSheet(req.query['link'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports.handler = handler;


[AsyncFunction: handler]

sheet marketing import?


In [10]:
var getDataSheet = importer.import('google sheet array objects');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';

function importSheet(docId, domain) {
    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(!match || domain !== match.domain && domain !== match.bucket) {
                throw new Error(`Sheet ${docId} doesn't match domain ${domain}`);
            }
            return getTemplates(docsId)
        })
        .then(t => {
            templates = t;
            key = Object.keys(templates).filter(k => templates[k].template)[0];
            return getTemplateProperties(key, properties, templates)
                .then(() => wrapTemplate(key, properties[key], properties));
        })
        .then(page => {
            // TODO: move this in to a new function for detecting links and writing every part of site
            return collectTemplateResources(key, page, properties, templates, domain);
        })
        .then(resources => {
            console.log(resources);
        })
}

module.exports = importSheet;


[Function: importSheet]

sheet marketing import handler?


In [ ]:
var importer = require('../Core');
var importSheet = importer.import('sheet marketing import');

async function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return await importSheet(req.query['link'], req.query['domain'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports.handler = handler;


setup sheet backend?


In [6]:
var getDataSheet = importer.import('google sheet array objects');
var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var addIP = importer.import('check dns');
var {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
} = importer.import('add google bucket web map');

var urlMap = 'web-map';

function setupBackend(docId, domain) {
    var project;

    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(!match || domain !== match.domain && domain !== match.bucket) {
                throw new Error(`Sheet ${docId} doesn't match domain ${domain}`);
            }
            project = match.project;
            return addIP(project, domain);
        })
        .then(ip => insertGlobalForward(project, ip, urlMap, domain))
        .then(() => insertBackendBucket(project, domain))
        .then(() => updateUrlMap(project, urlMap, domain))
}

module.exports = setupBackend;

reading notebook /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb
compiling /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb[4]
reading notebook /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb
compiling /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb[10]
reading notebook /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb
compiling /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb[0]
reading notebook /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb
compiling /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb[1]
reading notebook /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb
compiling /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb[2]
reading notebook /Users/briancullinan/jupytangular2/Cloud Services/google cloud api.ipynb
compiling /Users/briancullina

[Function: setupBackend]

sheet backend handler?


In [9]:
var importer = require('../Core');
var setupBackend = importer.import('setup sheet backend');

function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return setupBackend(req.query['link'], req.query['domain'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}
    
module.exports.handler = handler;


reading notebook /Users/briancullinan/jupytangular2/Marketing/convert spreadsheet.ipynb
compiling /Users/briancullinan/jupytangular2/Marketing/convert spreadsheet.ipynb[13]


[Function: handler]

package.json?


In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1"
    }
}
